# Práctico 5: Cliff Walking

## Dependencias

In [41]:
import gymnasium
import numpy as np
import random

In [42]:
# Cambiar a rgb_array para entrenar/testear

# env = gymnasium.make("CliffWalking-v0", render_mode="human")
env = gymnasium.make("CliffWalking-v0", render_mode="rgb_array")

Obtener la cantidad de estados y acciones

In [43]:
actions = env.action_space.n
states = env.observation_space.n

actions # Movimientos

np.int64(4)

Inicialización de la tabla Q

In [44]:
# Para cada estado tiene 4 valores, porque son 4 las acciones
Q = np.zeros((states, actions))
Q

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],


Obtención de la acción a partir de la tabla Q

In [45]:
def optimal_policy(state, Q):
    action = np.argmax(Q[state]) # Cual es el valor maximo que tengo en el estado en el que estoy parado.
    return action

Epsilon-Greedy Policy

In [46]:
def epsilon_greedy_policy(state, Q, epsilon=0.1):
    explore = np.random.binomial(1, epsilon)
    if explore:
        action = env.action_space.sample()
        print('explore')
    else:
        action = np.argmax(Q[state])
        print('exploit')
        
    return action

Ejemplo de episodio 

In [47]:
# obs,_ = env.reset()
# print(obs)
# done = False
# total_reward = 0
# step_count = 0
# while not done:
#     state = obs
#     action = epsilon_greedy_policy(state, Q, 0.5)
#     obs, reward, done, _, _ = env.step(action)
#     total_reward += reward
#     step_count += 1
#     env.render()
# print('total_reward', total_reward)
# print('total_steps', step_count)

# Ya teniendo el bucle este de episodios, tenemos que hacer algo similar que esto varias veces, actualizando el Q.
# Si tenemos mil episodios, va a demorar mucho. Más de 30 mins.
# Cuando vayamos a entrenar, el environmente no lo dejamos con render_mode human.
# El render, en vez de cambiarle el render mode, podemos directamente no renderizarlo y no llamar al .render.
# Lo que podemos hacer, renderizar por ejemplo, cada 100 episodios. Nos da una imagen visual de ver si está 
# aprendiendo. Es una forma facil de visualizar si está aprendiendo o no.
# El punto es, no hagan render siempre.
# Ver: https://medium.com/@lgvaz/understanding-q-learning-the-cliff-walking-problem-80198921abbc 
# 
# Precisamos haber pasado por el estado muchas veces para conocerlo.

In [ ]:
class QLearningAgent:
    def __init__(self, states, actions):
        self.q = np.zeros((states, actions)) # Inicializar arbitrariamente la tabla Q(s, a).
        self.train_stats_per_episode = [] # Inicializar arbitrariamente la tabla de estadísticas por episodio.
        self.test_stats_per_episode = [] # Inicializar arbitrariamente la tabla de estadísticas por episodio.

    def next_action(self, obs):
        # Decidir una accion para interactuar con el entorno. Te pregunta que es lo que vas a hacer.
        # Uso el que maximiza mi policy?
        return self.optimal_policy(obs)

    def train_agent(self, env, episodes=1000, epsilon=.9, gamma=.9, alpha=.99, episode_render_qty=100, epsilon_reduce_episode_qty=100, epsilon_reduce_episode_val=.09):
        # Episode: An episode refers to a sequence of actions taken by the agent in 
        # the environment until it reaches a terminal state.

        # Hiperparámetros: Estos son clave para que el aprendizaje sea efectivo:
        #   alpha (learning rate): qué tanto se ajustan los valores aprendidos.
        #   gamma (discount factor): cuánto valorás las recompensas futuras.
        #   epsilon (exploration rate): comenzá alto (explora más) y reducilo gradualmente.
        #   epsilon_decay = https://stackoverflow.com/questions/53198503/epsilon-and-learning-rate-decay-in-epsilon-greedy-q-learning
        current_epsilon = epsilon
        for episode in range(episodes):
            print('Episode', episode)
            # epsilon bajará en .09 cada 100 episodios (si dejamos los valores por defecto)
            if episode > 0 and (episode % epsilon_reduce_episode_qty == 0):
                current_epsilon = current_epsilon - epsilon_reduce_episode_val
            episode_stats = self.new_episode(env, current_epsilon, gamma, alpha)
            self.train_stats_per_episode.append(episode_stats) # Guardo estadisticas
            if (episode % episode_render_qty == 0):
                print("render")
                env.render()

    def test_agent(self, env, episodes=10):

        # Después de cierto número de episodios:
        # Probá al agente sin exploración (ε = 0) para ver si aprendió a comportarse bien.
        # Medí métricas: número de pasos por episodio, recompensa acumulada, tasa de éxito, etc.
        for episode in range(episodes):
            episode_stats = self.new_episode(env, 0, 0, 0)
            self.test_stats_per_episode.append(episode_stats) # Guardo estadisticas
            env.render()
        pass

    def new_episode(self, env, epsilon, gamma, alpha):
        # npe = Notificar con un print cada X steps
        s,_ = env.reset()
        # print(s) # s = obs
        done = False
        total_reward = 0
        step_count = 0
        while not done:
            a = self.epsilon_greedy_policy(env, s, epsilon)
            sN, r, done, _, _ = env.step(a)
            self.q[s][a] = self.q[s][a] + alpha * (r + gamma * np.max(self.q[sN]) - self.q[s][a])
            s = sN
            total_reward += r
            step_count += 1
            # if step_count % 1000 == 0:
            #     print('step_count', step_count)
        print('total_reward', total_reward)
        print('total_steps', step_count)
        return total_reward, step_count

    def epsilon_greedy_policy(self, env, state, epsilon):
        explore = np.random.binomial(1, epsilon)
        if explore:
            action = env.action_space.sample()
            #print('explore')
        else:
            action = np.argmax(self.q[state])
            #print('exploit')
        return action
    
    def optimal_policy(self, state):
        # Devuelve cual es el valor maximo que tengo en el estado en el que estoy parado.
        action = np.argmax(self.q[state])
        return action

In [49]:
agent = QLearningAgent(states, actions);
agent.train_agent(env)

Episode 0
total_reward -30612
total_steps 3288
render
Episode 1
total_reward -7918
total_steps 1186
Episode 2
total_reward -22941
total_steps 2646
Episode 3
total_reward -7555
total_steps 625
Episode 4
total_reward -27128
total_steps 2972
Episode 5
total_reward -22907
total_steps 2909
Episode 6
total_reward -21261
total_steps 2550
Episode 7
total_reward -8399
total_steps 974
Episode 8
total_reward -4186
total_steps 424
Episode 9
total_reward -5368
total_steps 517
Episode 10
total_reward -895
total_steps 202
Episode 11
total_reward -2041
total_steps 259
Episode 12
total_reward -10064
total_steps 1154
Episode 13
total_reward -3252
total_steps 282
Episode 14
total_reward -12715
total_steps 1429
Episode 15
total_reward -21219
total_steps 2508
Episode 16
total_reward -5581
total_steps 532
Episode 17
total_reward -4473
total_steps 711
Episode 18
total_reward -1266
total_steps 177
Episode 19
total_reward -1088
total_steps 197
Episode 20
total_reward -9513
total_steps 1296
Episode 21
total_rew

In [50]:
env = gymnasium.make("CliffWalking-v0", render_mode="human")
agent.test_agent(env)

total_reward -13
total_steps 13


NameError: name 'episode_render_qty' is not defined